In [ ]:
import numpy as np
import mrcfile
from matplotlib import pyplot as plt
%matplotlib inline
import collections
import tensorflow as tf
from tensorflow import spectral

In [ ]:
from relion_fixed_it import load_star

In [ ]:
sess = tf.InteractiveSession()
base_path = '/local/scratch/public/sl767/SPA/playing2/'
path=base_path+'Refine3D/1Q9J/1Q9J_mult005_it013_class001_external_reconstruct.star'

In [ ]:
file=load_star(path)

In [ ]:
type(file)

In [ ]:
for key, data in file.items():
    print(key)

In [ ]:
for key, data in file['external_reconstruct_general'].items():
    print(key)
    print(data)

In [ ]:
with mrcfile.open(base_path+file['external_reconstruct_general']['rlnExtReconsDataReal']) as mrc:
    data_real = mrc.data
with mrcfile.open(base_path+file['external_reconstruct_general']['rlnExtReconsDataImag']) as mrc:
    data_im = mrc.data
with mrcfile.open(base_path+file['external_reconstruct_general']['rlnExtReconsWeight']) as mrc:
    kernel = mrc.data

In [ ]:
complex_data = data_real + 1j * data_im

In [ ]:
complex_data.shape

In [ ]:
data = tf.placeholder(shape=(96, 96, 49), dtype=tf.complex64)
ft = spectral.irfft3d(data)

In [ ]:
image = sess.run(ft, feed_dict={data: complex_data})

In [ ]:
image.shape

In [ ]:
plt.imshow(image[...,40])

In [ ]:
plt.imshow(data_real[...,20])

In [ ]:
new_conv = np.fft.fftshift(complex_data, axes=(0, 1))
plt.imshow(np.real(new_conv[...,0]))
plt.clim(-1e7,1e7)

In [ ]:
image = sess.run(ft, feed_dict={data: new_conv})

In [ ]:
plt.imshow(image[...,0])

In [ ]:
image.shape

In [ ]:
new_conv.max()

In [ ]:
with mrcfile.open('/local/scratch/public/sl767/SPA/playing2/1Q9J.mrc') as mrc:
    molecule = mrc.data

In [ ]:
real_inp = tf.placeholder(shape=(96,96,96), dtype=tf.float32)
fourierData = spectral.rfft3d(real_inp)

In [ ]:
plt.imshow(molecule[...,20])

In [ ]:
fou = sess.run(fourierData, feed_dict={real_inp: molecule})

In [ ]:
plt.imshow(np.real(fou[...,20]))

In [ ]:
fou.shape

In [ ]:
plt.imshow(sess.run(ft, feed_dict={data:fou})[...,20])

In [ ]:
plt.imshow(np.real(np.fft.rfftn(molecule)[...,20]))

In [ ]:
plt.imshow(np.fft.fftshift(image)[...,50], clim=(0,600))

In [ ]:
plt.imshow(molecule[...,50], clim=(0,12))

In [ ]:
molecule.max()

In [ ]:
molecule.min()

In [ ]:
np.fft.fftshift(image).max()

In [ ]:
np.fft.fftshift(image).min()

In [10]:
import odl
from odl.contrib import tensorflow
import numpy as np
import tensorflow as tf

In [22]:
space = odl.uniform_discr([0, 0, 0], [96, 96 ,96], [96, 96, 96],
                                             dtype='float32')

In [23]:
class ifftshift_odl(odl.Operator):
    def _call(self, x):
        return space.element(np.fft.ifftshift(x))
    
    def __init__(self):
        super(ifftshift_odl, self).__init__(space, space, linear=True)

class fftshift_odl(odl.Operator):
    def _call(self, x):
        return space.element(np.fft.fftshift(x))
    
    def __init__(self):
        super(fftshift_odl, self).__init__(space, space, linear=True)
        
    @property
    def adjoint(self):
        return ifftshift_odl()
fftshift_tf = odl.contrib.tensorflow.as_tensorflow_layer(fftshift_odl())

In [6]:
x = np.ones(shape=(96,96,96))
y = fftshift_odl().adjoint(x)

In [ ]:
(ifftshift_odl()(fftshift_odl()(molecule))-molecule).__array__().max()

In [7]:
fft_tf = odl.contrib.tensorflow.as_tensorflow_layer(fftshift_odl())

In [20]:
FOURIER_SIZE = (None, 96, 96, 49,1)
t = tf.placeholder(shape=FOURIER_SIZE, dtype=tf.complex64)
y = tf.expand_dims(tf.spectral.irfft3d(t[...,0]), axis=-1)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
res = sess.run(y, feed_dict={t:np.ones(shape=(5,96,96,96,1))})

In [21]:
y.shape

TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(96), Dimension(1)])

In [24]:
y_sh = fftshift_tf(y)

In [25]:
y_sh.shape

TensorShape([Dimension(None), Dimension(96), Dimension(96), Dimension(96), Dimension(1)])